In [9]:
%pip install matplotlib
%pip install japanize-matplotlib
%pip install seaborn
%pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib 

ModuleNotFoundError: No module named 'distutils'

In [ ]:
# ① CSV のパス
csv_path = '/content/drive/MyDrive/Data/SSDSE-A-2024.csv'

# ② 先頭 1 行（不要な行）をスキップして読み込む
#    → 2 行目が列名、3 行目以降がデータとなる
df = pd.read_csv(
        csv_path,
        encoding='shift-jis',   # 元ファイルが Shift-JIS の場合
        skiprows=2,            # 先頭 1 行を読み飛ばす
        header=0               # （既定値）skiprows 後の 1 行目を列名にする
     )

# ③ これですでに DataFrame 型
print(type(df))         # <class 'pandas.core.frame.DataFrame'>

# ④ 内容を確認
print(df.head())        # 先頭 5 行を表示

# ⑤ 必要であればインデックスを振り直す
df.reset_index(drop=True, inplace=True)

# ⑥ 後で再保存したい場合
# df.to_csv(csv_path, index=False, encoding='shift-jis')